# Load Data

In [7]:
from tensorflow import keras

(train_data, train_targets), (test_data, test_targets) = keras.datasets.boston_housing.load_data()


57344/57026 [==============================] - 0s 1us/step


# Prepare Data

In [8]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

# Build Model


In [11]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(64, activation = 'relu', input_shape=(train_data.shape[1],)))
    model.add(keras.layers.Dense(64, activation= 'relu'))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer='rmsprop', loss = 'mse', metrics=['mae'])
    return model

# K Fold Validation



In [12]:
import numpy as np

k = 4
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[: i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis = 0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i + 1) * num_val_samples:]], axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs,
             batch_size = 1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets,  verbose = 0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [13]:
all_scores

[2.1671857833862305, 2.8652875423431396, 2.895418405532837, 2.373713970184326]